In [ ]:
# ================================================
# 0. CONFIG – change these if you want
# ================================================
N_PER_INDUSTRY = 2      # how many cheapest companies per industry
MAX_TICKERS = None      # set to a number (e.g. 300) to test on a subset, or None for all


# ================================================
# 1. Imports
# ================================================
import numpy as np
import pandas as pd
import yfinance as yf
from pathlib import Path
from tqdm import tqdm

tqdm.pandas()


# ================================================
# 2. Load ASX company list & fix header
# ================================================
csv_path = Path("ASXListedCompanies.csv")

# Detect header line that starts the actual table
lines = csv_path.read_text(encoding="utf-8").splitlines()
header_idx = None
for i, line in enumerate(lines):
    if line.startswith("Company name"):
        header_idx = i
        break

if header_idx is None:
    raise ValueError("Could not find header line starting with 'Company name'")

print(f"Header found at line index: {header_idx}")

# Load valid data from header onwards
df = pd.read_csv(csv_path, skiprows=header_idx)

# Optional: limit number of tickers for testing
if MAX_TICKERS is not None:
    df = df.head(MAX_TICKERS).copy()
    print(f"Using only first {len(df)} tickers for testing.")
else:
    print(f"Total tickers in dataset: {len(df)}")


# ================================================
# 3. Convert ASX code -> yfinance ticker
# ================================================
def to_yf_ticker(asx_code: str) -> str:
    code = str(asx_code).strip()
    if "." in code:
        return code
    return code + ".AX"

df["yf_ticker"] = df["ASX code"].apply(to_yf_ticker)


# ================================================
# 4. Function to fetch PE ratio from yfinance
# ================================================
def get_pe_ratio(ticker: str):
    try:
        info = yf.Ticker(ticker).info
        pe = info.get("trailingPE") or info.get("forwardPE")
        return float(pe) if pe is not None else None
    except Exception as e:
        # yfinance often throws 404s etc for some tickers – just log and skip
        print(f"Failed for {ticker}: {e}")
        return None


# ================================================
# 5. Fetch P/E ratios
# ================================================
print("Fetching P/E ratios... (this can take a while for the full dataset)")
df["pe_ratio"] = df["yf_ticker"].progress_apply(get_pe_ratio)

print("\nSample of raw PE data:")
display(df.head())


# ================================================
# 6. Clean P/E values
# ================================================
df["pe_ratio_clean"] = df["pe_ratio"].replace([np.inf, -np.inf], np.nan)
df.loc[df["pe_ratio_clean"] <= 0, "pe_ratio_clean"] = np.nan

filtered_df = df.dropna(subset=["pe_ratio_clean"]).copy()

print(f"\nValid P/E rows after cleaning: {len(filtered_df)}")


# ================================================
# 7. Compute industry average P/E (pivot table)
# ================================================
industry_pe = (
    filtered_df
    .groupby("GICS industry group")["pe_ratio_clean"]
    .mean()
    .reset_index()
    .sort_values("pe_ratio_clean")
    .rename(columns={"pe_ratio_clean": "avg_pe_ratio"})
)

print("\nIndustry average PE (pivot table):")
display(industry_pe)

# Save pivot table
pivot_output_path = "industry_average_pe_full_dataset.csv"
industry_pe.to_csv(pivot_output_path, index=False)
print(f"\nSaved pivot table to: {pivot_output_path}")


# ================================================
# 8. Merge company PE with industry averages
# ================================================
merged = filtered_df.merge(industry_pe, on="GICS industry group", how="left")

# Compute relative P/E (company PE / industry avg PE)
merged["pe_relative"] = merged["pe_ratio_clean"] / merged["avg_pe_ratio"]

merged = merged.dropna(subset=["pe_relative"])

print(f"\nMerged rows with valid relative PE: {len(merged)}")


# ================================================
# 9. Sort by industry + relative PE, then take N per industry
# ================================================
merged_sorted = merged.sort_values(
    ["GICS industry group", "pe_relative"],
    ascending=[True, True]
)

topN_per_industry = merged_sorted.groupby("GICS industry group").head(N_PER_INDUSTRY)

print(f"\nTop {N_PER_INDUSTRY} CHEAPEST companies per industry:")
display(topN_per_industry[[
    "GICS industry group",
    "Company name",
    "yf_ticker",
    "pe_ratio_clean",
    "avg_pe_ratio",
    "pe_relative"
]])


# ================================================
# 10. Save tickers ONLY to tickers.csv
# ================================================
tickers_only = topN_per_industry[["yf_ticker"]].rename(columns={"yf_ticker": "ticker"})
tickers_only.to_csv("tickers.csv", index=False)

print("\nSaved tickers.csv with the selected tickers:")
display(tickers_only.head())


# ================================================
# DONE – only TWO CSVs are saved:
# 1. industry_average_pe_full_dataset.csv
# 2. tickers.csv
# ================================================


Header found at line index: 2
Total tickers in dataset: 1985
Fetching P/E ratios... (this can take a while for the full dataset)


  2%|█▊                                                                              | 45/1985 [00:47<36:08,  1.12s/it]